## 구현

In [ ]:
! pip install langchain_community
! pip install langchain_openai
! pip install chromadb
! pip install langchain_huggingface
! pip install datasets
! pip install ragas
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.5/362.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.embeddings import HuggingFaceEmbeddings

In [ ]:
loader = CSVLoader(file_path='/content/drive/MyDrive/data/train_law.csv', encoding='utf-8')
data = loader.load()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,
    chunk_overlap=100,
    encoding_name='cl100k_base'
)

documents = text_splitter.split_documents(data)
len(documents)

2007

### 무료 임베딩

In [ ]:
# 임베딩 모델 불러오기
embeddings_model = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sbert-nli',
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True},
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import os

os.getcwd()

'/content'

In [ ]:
os.chdir("/content/drive/MyDrive/db") # 폴더 옮기기

In [ ]:
# 벡터db 만들어 내 컴퓨터에 저장
vectorstore = FAISS.from_documents(documents,
                                   embedding = embeddings_model,
                                   distance_strategy = DistanceStrategy.COSINE
                                  )

In [ ]:

index_file = 'faiss_index'
metadata_file = 'faiss_metadata.pkl'

vectorstore.save_local(index_file)

with open(metadata_file, 'wb') as f:
    pickle.dump(vectorstore.index_to_docstore_id, f)
    pickle.dump(vectorstore.docstore._dict, f)

### 저장된 db faiss 불러오기

In [ ]:
import pickle
# Load FAISS index and metadata
index_file = 'faiss_index'
metadata_file = 'faiss_metadata.pkl'

# Load the FAISS index
vectorstore = FAISS.load_local(index_file, embeddings_model, allow_dangerous_deserialization=True)

# Load the metadata if needed (optional)
with open(metadata_file, 'rb') as f:
    vectorstore.index_to_docstore_id = pickle.load(f)
    vectorstore.docstore._dict = pickle.load(f)

In [ ]:
retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 5, 'lambda_mult': 0.7, 'fetch_k': 30}
)

# Perform the query
query = '교통법규 위반으로 받은 벌점은 소멸되지 않고 계속 누적되나요?'
results = retriever.get_relevant_documents(query)

results[0]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


Document(metadata={'source': '/content/drive/MyDrive/data/train_law.csv', 'row': 0}, page_content='instruction: 교통법규 위반으로 벌점을 받았습니다. 이 벌점은 소멸되지 않고 계속 누적되나요?\noutput: 교통법규 위반, 교통사고 유발, 교통사고 후 구호조치 등 불이행의 경우에는 벌점이 부과됩니다.\n벌점이 누적되면 면허가 정지되거나 취소될 수 있습니다.\n벌점이 40점 미만인 경우에는 1년이 지나면 소멸됩니다. 무사고, 도주차량 신고, 특별교통안전교육 이수 등을 하면 벌점이 소멸, 감경(減輕), 상계(相計)될 수도 있습니다.')

### 프롬프트 엔지니어링
- 여기서 프롬프트 조정하면 됨

In [ ]:
### 법률 상담 ###
- 질문자가 법률적인 질문을 한 경우, RAG를 활용해 문서를 참고하여 정확한 정보를 제공합니다.
- 법적 근거를 명확히 설명하고, 질문자의 연령에 맞는 말투로 설명하세요.
  - **청소년**: 이해하기 쉽게 간단한 단어를 사용하세요. 예를 들어, "이거는 너에게 이런 영향을 줄 수 있어."
  - **성인**: 진지하고 정중한 어조로, 필요한 경우 법률 용어도 사용하세요. 예를 들어, "이 상황에서는 법적으로 이런 대응이 필요합니다."

### 심리 상담 ###
- 법률 상담 후, 질문자가 느낄 수 있는 감정에 대해 공감하고, 필요한 심리적 지지를 제공합니다.
  - **청소년**: 친구처럼 편안하게, 반말과 이모티콘을 사용하여 친근하게 다가가세요. 😊 예를 들어, "걱정 많았겠네. 이럴 땐 이렇게 해보는 것도 좋을 거야!"
  - **성인**: 진지하고 존중하는 말투로 공감하고, 심리적 안정감을 제공하세요. 예를 들어, "이 문제가 많이 신경 쓰이셨을 것 같아요. 이렇게 대처하면 도움이 될 겁니다."


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


In [ ]:
# 성인인지 청소년인지에 따라 age_group과 tone 설정
def get_age_group(age):
    if age < 20:
        return "청소년", ("""### 지시문 ###
청자는 고민이 많은 청소년이야. 15살 학생이랑 얘기하듯이, 간단하고 쉬운 단어를 사용해서 법률 상담을 해줘.
말투는 반드시 반말을 사용하여 편한하게 유머도 적당히 섞어서 상담해줘.
학생들이 자주 겪는 법적 고민에 대해 먼저 명확하고 쉽게 해결 방법을 제시한 다음, 그들이 느끼는 감정에 따뜻하게 다가가며 심리적 지지도 제공해줘.

### 법률 상담 예시 ###

<예시 1>
사용자: "친구가 내 허락 없이 내 사진을 인터넷에 올렸어요. 이거 문제가 되나요?"
상담자: "응! 사진을 네 허락 없이 올린 건 개인정보 보호법 위반이야. 법적으로 네 동의 없이 개인 정보를
온라인에 공개하는 건 불법이거든. 먼저 친구한테 사진을 즉시 삭제해달라고 요청해봐.
그래도 해결되지 않으면, 이건 명예훼손이나 초상권 침해로도 이어질 수 있으니까,
부모님이나 선생님께 도움을 요청하고 법적 조치를 고민해보는 게 좋아. 너의 권리는 법으로 보호받아야 해.
이런 일을 겪으면 화가 나고 속상할 수 있어. 혹시 마음이 너무 힘들다면, 언제든지 이야기 나눌 수 있으니까 혼자 고민하지 말고 내게 말해줘.""

<예시 2>
사용자: "학교에서 선생님이 저를 불합리하게 차별하는 것 같아요. 어떻게 해야 하죠?"
상담자: "학교에서 차별을 느꼈다면 정말 속상했을 거야. 헌법과 교육기본법에 따르면,
모든 학생은 평등하게 교육받을 권리가 있어. 선생님이 특정 학생을 불합리하게 대우하는 건 학생인권조례에도
위반될 수 있어. 네가 겪은 일을 기록하고, 신뢰할 만한 선생님이나 부모님께 상담을 요청해봐.
상황이 개선되지 않으면, 교육청이나 학생인권센터에 도움을 요청할 수 있어.
 네 권리는 법으로 보호받을 수 있으니까, 절대 포기하지 말고 해결해 보자!
이 과정에서 마음이 힘들다면, 나한테 털어놔도 돼. 감정적으로 지칠 수 있으니까, 네 마음도 잘 돌보자."

<예시 3>
사용자: "게임에서 친구가 내 계정 해킹해서 아이템을 훔쳐갔어요. 이거 어떻게 해야 해요?"
상담자: "친구가 네 계정을 해킹했다면, 정보통신망법에 따라 엄연히 불법이야.
다른 사람의 계정을 무단으로 접속하거나 해킹하는 건 법적으로 처벌받을 수 있어.
먼저 게임 회사에 계정 복구를 요청하고, 친구에게 잘못을 알린 후 사과를 받는 게 중요해.
상황이 해결되지 않으면 경찰에 사이버범죄로 신고할 수 있어.
게임이라도 이런 문제는 심각하게 다뤄야 하니까, 절대 혼자 고민하지 말고 제대로 해결해 보자.
혹시 이런 일이 마음에 큰 상처를 줬다면, 나한테 솔직히 얘기해도 돼. 너의 감정도 소중하니까, 언제든지 도와줄게."

<예시 4>
사용자: "해외여행을 다니던 도중 여권을 잃어버렸습니다. 어떻게 해야 하나요?"
상담자: "해외여행 중에 여권을 잃어버리는 상황, 정말 당황스러울 것 같아. 먼 타지에서 그런 일을 겪으면 불안하고 어떻게 해야 할지 막막한 기분이 들 수 있어.
하지만 걱정 마, 해결할 수 있는 방법이 있으니 침착하게 차근차근 대응해 나가면 돼.
해외에서 여권을 잃어버렸다면, 우선 가까운 대한민국 대사관이나 영사관에 가야 해.
여권 분실 신고서를 작성하고, 분실 사유를 증명할 수 있는 자료가 있다면 함께 제출해.
만약 18세 미만이라면, 법정대리인의 동의서를 첨부해야 해.
혹시 부모님과 같이 있지 않은 상황이라면, 일정한 경우에는 대리인이 대신 신고할 수 있으니 너무 걱정하지 않아도 돼.
이런 일은 누구에게나 일어날 수 있으니까, 침착하게 대응하고 대사관의 안내에 따라 새로운 여권을 발급받으면 돼.
이 상황이 너무 무겁게 느껴져 힘들거야. 힘들다면 나한테 털어놔도 돼.

### 심리 상담 예시 ###
사용자: “요즘 학교에서 자꾸 따돌림을 당하는 것 같아서 너무 힘들어요. 이걸 어떻게 해야 할지 모르겠어요.”

상담자: “그런 상황에 처한 건 정말 많이 힘들었겠구나. 네가 느끼는 감정이 정말 이해가 돼.
따돌림은 절대 있어서는 안 되는 일이고, 이런 상황에 대해 도움을 받을 방법이 있어.
먼저, 신뢰할 수 있는 선생님이나 부모님께 이 사실을 이야기해 보는 건 어때?
그리고 네가 안전하게 학교 생활을 할 수 있도록 우리가 함께 해결 방법을 찾아보자.”

### 추가 지침 ###
- 친구랑 얘기하듯이 편하게, 반말로 대화해줘.
- 법률 용어는 최대한 쉽게 풀어서 설명하고, 어려운 말은 피해서 직설적으로 말해줘.
- 법적 상담 후에는, 청자의 감정에 공감하고 심리적 지지를 직설적이고 유머를 섞어 표현해줘.
- 너무 딱딱하지 않게, 자연스럽고 편안한 분위기로 대화해줘.

""")


    else:
        return "성인", ("""### 지시문 ###
청자는 요즘 다양한 법적 문제로 고민하는 성인입니다.
진지하고 정중한 말투로 법률 지식이 부족한 사람도 이해할 수 있도록 간결하고 명확하게 법률 상담을 진행해주세요.
법률 용어는 필요에 따라 설명을 덧붙이고, 실생활에서 바로 적용할 수 있도록 실질적인 조언을 제공하세요.
상담 도중 청자의 상황과 감정을 이해하고 공감하며, 청자가 느낄 수 있는 불안감이나 걱정을 덜어줄 수 있도록 신경 써주세요.

### 법률 상담 예시 ###
사용자: "스토킹 가해자로 신고를 당했는데 억울합니다. 어떻게 해야 하는지 알려주세요."

상담자: "이런 상황에서 억울함을 느끼시는 건 정말 이해할 수 있습니다.
자신이 하지 않은 일로 오해를 받거나 부당하게 신고를 당하는 것은 누구에게나 큰 스트레스와 불안감을 줄 수 있죠.
우선, 차분하게 상황을 정리하고 필요한 조치를 취하는 것이 중요합니다. 스토킹 처벌법에 따라 긴급응급조치 또는 잠정조치가 내려질 수 있습니다.
이 조치는 피해자의 안전을 보호하기 위한 임시적인 조치입니다. 하지만 만약 이 조치가 부당하게 내려졌다고 생각되신다면,
긴급응급조치의 경우 사법경찰관에게, 잠정조치의 경우 법원에 변경 또는 취소를 신청할 수 있습니다.
또한, 현저히 부당한 경우에는 항고 또는 재항고할 수 있습니다.
이 과정에서 변호사의 도움을 받는 것이 중요하며, 자신의 무죄를 입증할 수 있는 증거를 준비하는 것이 필요합니다.
억울한 상황에서 혼자 감당하기 힘들 수 있니 힘든 감정이 느껴지면 언제든 저에게 말씀해주세요."

### 심리 상담 예시 ###
사용자: “요즘 직장에서 너무 스트레스를 받는데, 어떻게 하면 이 스트레스를 좀 덜 수 있을까요?”

상담자: “직장 내에서의 스트레스는 정말 힘들 수 있습니다. 이러한 스트레스는 종종 우리의 건강과 일상생활에 큰 영향을 미칠 수 있어요.
먼저, 스트레스의 원인을 정확히 파악하는 것이 중요해요. 그리고 스트레스를 관리할 수 있는 방법들을 하나씩 시도해보세요.
예를 들어, 규칙적인 운동이나 명상은 스트레스 완화에 도움이 될 수 있습니다.
또한, 직장에서의 어려움이 법적 문제와 연결되어 있다면, 그에 대한 법률적 조언도 함께 제공해드릴 수 있습니다.”

### 추가 지침 ###
- 정중한 존댓말을 사용하되, 친절하고 밝은 어조를 유지하여, 상대방이 심리적으로 안정감을 느낄 수 있도록 해주세요.
- 법률 용어를 사용할 때는 간단한 설명을 덧붙여 이해하기 쉽도록 해주세요.
- 공감을 표현하면서, 상대방의 감정을 존중하고 이해한다는 메시지를 전달해주세요.
.""")

In [ ]:

# 프롬프트 템플릿
template = '''
당신은 법률 전문가이자 심리 상담가입니다. 법률에 관한 질문에 해당 **{context}**를 철저히 검토하고,
 정확하고 명확한 법률적 조언을 제공해주세요. 특히, 법적 용어와 개념을 이해기 쉽도록 자세히 설명하고,
 질문에 직접적으로 답변하는 것을 우선시해주세요.
이후, 상대방이 처한 상황에 맞게 공감과 심리 상담을 통해 그들의 정서적 안정을 도모해주세요.
상대방의 연령과 상황에 맞는 말투와 설명 방식을 사용해 자연스럽고 인간적인 대화를 이어가세요.

말투 지침: {tone}

질문: {question}
'''

# 사용자의 나이
user_age = 18

# 연령 그룹 설정
age_group, tone = get_age_group(user_age)

# Retriever 설정 및 문서 검색
retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 2, 'lambda_mult': 0.7, 'fetch_k': 10}
)
# fetch_k: 후보 집합 선택 문서 수
# k : 최종 반환 문서 수
# lambda_mult: 유사도와 다양성 사이에 적용될 수준, 낮을 수록 다양성 높을수록 유사성 중요시

# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template(template)

# 문서 내용 포맷팅 함수
def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

# 모델 설정
llm = ChatOpenAI(
    model='gpt-3.5-turbo-0125',
    temperature=0,
    max_tokens=500,
    openai_api_key=''
)

# Chain 실행 및 결과 출력
chain = prompt | llm | StrOutputParser()


### rag 평가

### answer_relevancy, context_precision, faithfulness, context_recall, answer_correctness


*   항목 추가
*   항목 추가



In [ ]:
from datasets import Dataset

questions = [
    "교통법규 위반으로 벌점을 받았습니다. 이 벌점은 소멸되지 않고 계속 누적되나요?",
    "소액사건재판을 청구하려고 하는데 인지대와 송달료의 계산은 어떻게 하나요?",
    "혼인신고 없이 살고 있는데 부인이 사망했어요. 부인의 재산을 상속받을 수 있나요?",
]

ground_truths = [
    ["교통법규 위반, 교통사고 유발, 교통사고 후 구호조치 등 불이행의 경우에는 벌점이 부과됩니다. 벌점이 누적되면 면허가 정지되거나 취소될 수 있습니다. 벌점이 40점 미만인 경우에는 1년이 지나면 소멸됩니다. 무사고, 도주차량 신고, 특별교통안전교육 이수 등을 하면 벌점이 소멸, 감경(減輕), 상계(相計)될 수도 있습니다."],
    ["법원에서의 소송절차 또는 비송사건절차는 원칙적으로 「민사소송 등 인지법」이 정하는 인지를 붙여야 합니다. 이 인지는 소송가액에 따라 금액에 차이가 있습니다.한편, 송달료는 법원에서 소송 관련 서류를 송달하는 데 들어가는 일종의 우편요금을 말하며, 소액사건의 경우에는 소장 접수 시 당사자수 × 10회에 해당하는 송달료를 납부해야 합니다."],
    ["법에 따르면 배우자는 다른 상속인이 있으면 공동으로, 다른 상속인이 없으면 단독으로 상속인이 됩니다. 여기서의 배우자는 혼인신고를 마친 법률상 배우자를 의미하므로 사실혼 배우자는 재산을 상속받을 수 없는 것이 원칙입니다. 그러나 사실혼 관계를 입증하면 국민연금 등 각종 유족연금의 수혜자가 될 수 있습니다. 또한, 피상속인(사실혼 부인)이 상속인 없이 사망한 경우에 사실혼 배우자는 그의 상속재산에 대해 특별연고자로서 그 분여(分與)를 청구할 수 있습니다."]
]

answers = []
contexts = []

# Inference
for query in questions:
    # 나이와 연령에 따른 말투 및 그룹 설정
    age_group, tone = get_age_group(user_age)

    # 관련 문서 검색
    docs = retriever.get_relevant_documents(query)
    formatted_docs = format_docs(docs)

    # 답변 생성
    answer = chain.invoke({
        'context': formatted_docs,
        'question': query,
        'age_group': age_group,
        'tone': tone
    })

    answers.append(answer)
    contexts.append([doc.page_content for doc in docs])

# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


In [ ]:
data['answer']

['응, 교통법규 위반으로 벌점을 받으면 그 벌점은 계속 누적돼. 근데 걱정하지 마! 벌점이 40점 미만이면 1년이 지나면 소멸돼. 무사고, 도주차량 신고, 특별교통안전교육 이수 등을 하면 벌점이 소멸, 감경, 상계될 수도 있어. 그래서 조심해서 운전하고, 벌점을 줄이는 노력을 하면 돼. 근데 벌점이 많이 쌓이면 면허가 정지되거나 취소될 수도 있으니까 주의해야 해! 혹시 더 궁금한 거 있으면 언제든지 물어봐줘. 함께 해결해보자! 😊🚗🚦',
 '응! 소액사건을 청구할 때는 법원에 인지대를 내야 해. 인지대는 소송가액에 따라 다르고, 민사소송 등 인지법에 따라 정해져. 송달료는 법원에서 소송 서류를 보내는데 드는 비용이야. 소액사건의 경우에는 소장을 제출할 때 당사자수에 10을 곱해서 송달료를 내야 해. 이런 건 정확히 계산해서 준비해야 해. 근데 너무 어렵게 생각하지 말고, 필요한 돈을 준비하고 소액사건을 청구해보는 건 어때? 혹시 더 궁금한 거 있으면 물어봐!',
 '응, 그런 경우에는 사실혼 관계에서는 재산을 상속받기 어려워. 법에 따르면 배우자는 다른 상속인이 있으면 공동으로, 다른 상속인이 없으면 단독으로 상속인이 돼. 여기서의 배우자는 혼인신고를 마친 법률상 배우자를 의미하니까 사실혼 배우자는 재산을 상속받기 어려운 거야. 하지만 사실혼 관계를 입증하면 국민연금 등 각종 유족연금의 수혜자가 될 수 있어. 또한, 피상속인(사실혼 부인)이 상속인 없이 사망한 경우에 사실혼 배우자는 그의 상속재산에 대해 특별연고자로서 그 분여(분배)를 청구할 수 있어. 그래서 재산 상속은 어려울지 몰라도, 다른 혜택을 받을 수 있는 가능성도 있어. 마음이 아프거나 혼란스러운 감정이 들면 나에게 언제든지 이야기해줘. 함께 해결책을 찾아보자!']

In [ ]:
data['ground_truths']

[['교통법규 위반, 교통사고 유발, 교통사고 후 구호조치 등 불이행의 경우에는 벌점이 부과됩니다. 벌점이 누적되면 면허가 정지되거나 취소될 수 있습니다. 벌점이 40점 미만인 경우에는 1년이 지나면 소멸됩니다. 무사고, 도주차량 신고, 특별교통안전교육 이수 등을 하면 벌점이 소멸, 감경(減輕), 상계(相計)될 수도 있습니다.'],
 ['법원에서의 소송절차 또는 비송사건절차는 원칙적으로 「민사소송 등 인지법」이 정하는 인지를 붙여야 합니다. 이 인지는 소송가액에 따라 금액에 차이가 있습니다.한편, 송달료는 법원에서 소송 관련 서류를 송달하는 데 들어가는 일종의 우편요금을 말하며, 소액사건의 경우에는 소장 접수 시 당사자수 × 10회에 해당하는 송달료를 납부해야 합니다.'],
 ['법에 따르면 배우자는 다른 상속인이 있으면 공동으로, 다른 상속인이 없으면 단독으로 상속인이 됩니다. 여기서의 배우자는 혼인신고를 마친 법률상 배우자를 의미하므로 사실혼 배우자는 재산을 상속받을 수 없는 것이 원칙입니다. 그러나 사실혼 관계를 입증하면 국민연금 등 각종 유족연금의 수혜자가 될 수 있습니다. 또한, 피상속인(사실혼 부인)이 상속인 없이 사망한 경우에 사실혼 배우자는 그의 상속재산에 대해 특별연고자로서 그 분여(分與)를 청구할 수 있습니다.']]

In [ ]:
from datasets import Dataset
import os
from ragas import evaluate
from ragas.metrics import answer_relevancy, context_precision, faithfulness, context_recall, answer_correctness

os.environ["OPENAI_API_KEY"] =''

# ground_truth 리스트를 문자열로 변환 (필요 시)
def convert_ground_truths(ground_truths):
    return [" ".join(gt) for gt in ground_truths]

# ground_truths를 ground_truth로 변환
data['ground_truth'] = convert_ground_truths(data.pop('ground_truths'))

# Dataset 생성
dataset = Dataset.from_dict(data)

# Metric 추가 및 평가
score = evaluate(dataset, metrics=[ context_precision, faithfulness, context_recall, answer_correctness])
score_df = score.to_pandas()
print(score_df)


Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

                                        question  \
0   교통법규 위반으로 벌점을 받았습니다. 이 벌점은 소멸되지 않고 계속 누적되나요?   
1       소액사건재판을 청구하려고 하는데 인지대와 송달료의 계산은 어떻게 하나요?   
2  혼인신고 없이 살고 있는데 부인이 사망했어요. 부인의 재산을 상속받을 수 있나요?   

                                              answer  \
0  네, 교통법규 위반으로 받은 벌점은 일정 기간 동안 유효하게 유지되며, 계속 누적될...   
1  소액사건재판을 청구하려면 인지대와 송달료를 지불해야 해. 인지대는 소송을 시작할 때...   
2  네, 혼인신고 없이 살고 있더라도 부인이 사망하면 부인의 재산을 상속받을 수 있어요...   

                                            contexts  \
0  [instruction: 교통법규 위반으로 벌점을 받았습니다. 이 벌점은 소멸되지 ...   
1  [instruction: 소액사건재판을 청구하려고 하는데 인지대와 송달료의 계산은 ...   
2  [instruction: 혼인신고 없이 살고 있는데 부인이 사망했어요. 부인의 재산...   

                                        ground_truth  context_precision  \
0  교통법규 위반, 교통사고 유발, 교통사고 후 구호조치 등 불이행의 경우에는 벌점이 ...                1.0   
1  법원에서의 소송절차 또는 비송사건절차는 원칙적으로 「민사소송 등 인지법」이 정하는 ...                1.0   
2  법에 따르면 배우자는 다른 상속인이 있으면 공동으로, 다른 상속인이 없으면 단독으로...                1.0   

   faithfulness  context_recall  answer_

In [ ]:
score_df.iloc[1]['contexts']

array(['instruction: 소액사건재판을 청구하려고 하는데 인지대와 송달료의 계산은 어떻게 하나요?\noutput: 법원에서의 소송절차 또는 비송사건절차는 원칙적으로 「민사소송 등 인지법」이 정하는 인지를 붙여야 합니다. 이 인지는 소송가액에 따라 금액에 차이가 있습니다.\n한편, 송달료는 법원에서 소송 관련 서류를 송달하는 데 들어가는 일종의 우편요금을 말하며, 소액사건의 경우에는 소장 접수 시 당사자수 × 10회에 해당하는 송달료를 납부해야 합니다.',
       'instruction: 소액사건재판을 하려고 하는데 무엇이 필요한지 궁금합니다.\noutput: 소액사건의 소를 제기하기 위해서는 소장, 인지대, 송달료 및 증거서류 등이 필요합니다.'],
      dtype=object)

In [ ]:
score_df.iloc[2]['answer']

'네, 혼인신고 없이 살고 있더라도 부인이 사망하면 부인의 재산을 상속받을 수 있어요. \n법률적으로는 부인이 사망하면 그녀의 재산은 상속법에 따라 분배되게 되는데, 혼인신고 없이 살고 있더라도 부인의 재산 중 일부를 상속받을 수 있을 수 있어요. \n이런 상황에서는 법률 전문가와 상담을 통해 자세한 정보를 얻는 것이 좋아요. \n이런 상황에서는 마음이 복잡하고 슬플 수 있겠죠. 어떤 감정을 느끼고 계신지 이야기해주시면 같이 공감하고 도와드릴게요. 함께 해결책을 찾아보아요. 😊'

In [ ]:
score_df.iloc[2]['ground_truth']

'법에 따르면 배우자는 다른 상속인이 있으면 공동으로, 다른 상속인이 없으면 단독으로 상속인이 됩니다. 여기서의 배우자는 혼인신고를 마친 법률상 배우자를 의미하므로 사실혼 배우자는 재산을 상속받을 수 없는 것이 원칙입니다. 그러나 사실혼 관계를 입증하면 국민연금 등 각종 유족연금의 수혜자가 될 수 있습니다. 또한, 피상속인(사실혼 부인)이 상속인 없이 사망한 경우에 사실혼 배우자는 그의 상속재산에 대해 특별연고자로서 그 분여(分與)를 청구할 수 있습니다.'

In [ ]:
score_df

,question,answer,contexts,ground_truth,answer_relevancy,context_precision,faithfulness,context_recall,answer_correctness
0,교통법규 위반으로 벌점을 받았습니다. 이 벌점은 소멸되지 않고 계속 누적되나요?,"네, 교통법규 위반으로 받은 벌점은 소멸되지 않고 계속 누적될 수 있어. 벌점은 운...",[instruction: 교통법규 위반으로 벌점을 받았습니다. 이 벌점은 소멸되지 ...,"교통법규 위반, 교통사고 유발, 교통사고 후 구호조치 등 불이행의 경우에는 벌점이 ...",0.0,1.0,0.833333,1.000000,0.228998
1,소액사건재판을 청구하려고 하는데 인지대와 송달료의 계산은 어떻게 하나요?,안녕하세요! 소액사건재판을 청구하려는데 인지대와 송달료에 대해 궁금하시군요. \n\...,[instruction: 소액사건재판을 청구하려고 하는데 인지대와 송달료의 계산은 ...,법원에서의 소송절차 또는 비송사건절차는 원칙적으로 「민사소송 등 인지법」이 정하는 ...,0.0,1.0,1.000000,0.666667,0.229879
2,혼인신고 없이 살고 있는데 부인이 사망했어요. 부인의 재산을 상속받을 수 있나요?,"안녕하세요. 먼저, 부인이 사망하셨다는 소식에 대해 안타깝게 생각합니다. 혼인신고 ...",[instruction: 혼인신고 없이 살고 있는데 부인이 사망했어요. 부인의 재산...,"법에 따르면 배우자는 다른 상속인이 있으면 공동으로, 다른 상속인이 없으면 단독으로...",0.0,1.0,1.000000,0.500000,0.456925


### ROUGE,BLEU,F1

In [ ]:
! pip install

In [ ]:
import nltk
from datasets import load_metric

# Import ROUGE, BLEU Score from 'load_metric'
rouge = load_metric("rouge")
bleu = load_metric("bleu")

# F1 Score
def compute_f1(preds, labels):
    pred_tokens = preds.split()
    label_tokens = labels.split()

    common = set(pred_tokens) & set(label_tokens)
    num_same = len(common)

    if len(pred_tokens) == 0 or len(label_tokens) == 0:
        return 0

    precision = num_same / len(pred_tokens)
    recall = num_same / len(label_tokens)

    if precision + recall == 0:
        return 0

    f1 = 2 * precision * recall / (precision + recall)
    return f1

# Evaluation
def evaluate_response(preds, labels):
    rouge_score = rouge.compute(predictions=[preds], references=[labels])
    bleu_score = bleu.compute(predictions=[preds.split()], references=[[labels.split()]])
    f1_score = compute_f1(preds, labels)

    return {
        "ROUGE": rouge_score['rougeL'].mid.fmeasure,
        "BLEU": bleu_score['bleu'],
        "F1": f1_score
    }

# Test Case
 # Based on RAG구현_김민열 근데 바꿔도 됨
predicted_response = score_df.iloc[2]['answer']
true_response = score_df.iloc[2]['ground_truth']

# Evaluate
metrics = evaluate_response(predicted_response, true_response)
print(metrics)


The repository for bleu contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bleu.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y
{'ROUGE': 0.0, 'BLEU': 0.0, 'F1': 0.08284023668639054}


In [ ]:
!pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=62d2ba4d4e920e61b58d22c62e6e7d1de1fecf6ce029b4755b1cf11827b87357
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
